# See how well it did on real data
The spectra are real. We have results from `ssv` files from SpectrumMill. We can load this and see if the sequences we identified matches the sequences SpectrumMill identified, plus the hybrid ones

## 1. Load the `ssv` file

In [1]:
import pandas as pd

specmilresultsfile = '/Users/zacharymcgrath/Downloads/NOD2_E3_results.ssv'
specmilresults = pd.read_csv(specmilresultsfile, sep=';')


In [2]:
specmilresults.head(2)

,number,filename,parent_charge,score,deltaForwardReverseScore,deltaRank1Rank2Score,percent_scored_peak_intensity,totalIntensity,previous_aa,sequence,next_aa,retentionTimeMin,chromatographicPeakWidthSec,parent_m_over_z,species,entry_name
0,1,NOD2_E3.13446.13477.2,2,10.10,10.10,9.91,84.5,183000.0,(E),DPQVEQLEL,(-),48.35,26.0,535.7725,MOUSE,ins1C18
1,2,NOD2_E3.18005.18246.2,2,12.84,11.07,12.84,97.8,40000000.0,(G),DLQTLALEVA,(-),65.78,29.0,536.8007,MOUSE,ins1C3


## 2. Load the results from hypedsearch

In [3]:
nonhybridresultsfile = '/Users/zacharymcgrath/Desktop/Experiment output/NOD2_E3/summary.tsv'
hybridresultsfile = '/Users/zacharymcgrath/Desktop/Experiment output/NOD2_E3/hybrid_summary.tsv'

nonhybridresults = pd.read_csv(nonhybridresultsfile, sep='\t')
hybridresults = pd.read_csv(hybridresultsfile, sep='\t')


In [4]:
nonhybridresults.head(10)

,Unnamed: 0,proteins,sequence,b_score,y_score,total_score,precursor_distance,entry name
0,0,"['T22D1_MOUSE TSC22 domain family protein 1', ...",PPPS,0.06,0.02,67,197.107719,/Users/zacharymcgrath/Desktop/nod2 data/NOD2_E...
1,1,[],GGISP,0.06,0.02,75,213.618451,/Users/zacharymcgrath/Desktop/nod2 data/NOD2_E...
2,2,['CALR_MOUSE Calreticulin'],GLAAA,0.02,0.02,50,199.620994,/Users/zacharymcgrath/Desktop/nod2 data/NOD2_E...
3,3,['KCNQ5_MOUSE Potassium voltage-gated channel ...,PKPAA,0.02,0.02,50,240.149918,/Users/zacharymcgrath/Desktop/nod2 data/NOD2_E...
4,4,[],VSVPA,0.08,0.04,76,234.641926,/Users/zacharymcgrath/Desktop/nod2 data/NOD2_E...
5,5,[],AASPA,0.10,0.02,76,206.610626,/Users/zacharymcgrath/Desktop/nod2 data/NOD2_E...
6,6,"['DAP1_MOUSE Death-associated protein 1', 'PCX...",PPAA,0.06,0.02,67,176.102437,/Users/zacharymcgrath/Desktop/nod2 data/NOD2_E...
7,7,['T22D1_MOUSE TSC22 domain family protein 1'],PPPAA,0.06,0.02,75,224.628819,/Users/zacharymcgrath/Desktop/nod2 data/NOD2_E...
8,8,"['T22D1_MOUSE TSC22 domain family protein 1', ...",AAA,0.10,0.02,102,114.568230,/Users/zacharymcgrath/Desktop/nod2 data/NOD2_E...
9,9,['RLA1_MOUSE 60S acidic ribosomal protein P1'],AAGAA,0.02,0.02,50,178.597519,/Users/zacharymcgrath/Desktop/nod2 data/NOD2_E...


In [5]:
hybridresults.head(2)

,Unnamed: 0,left_proteins,right_proteins,sequence,hybrid_sequence,b_score,y_score,total_score,precursor_distance,entry name
0,0,['TNFL9_MOUSE Tumor necrosis factor ligand sup...,['CBPD_MOUSE Carboxypeptidase D'],TTLNWALEAAAA,TTLNW-ALEAAAA,0.04,0.04,28,101.683426,/Users/zacharymcgrath/Desktop/nod2 data/NOD2_E...
1,1,['SARNP_MOUSE SAP domain-containing ribonucleo...,['LYST_MOUSE Lysosomal-trafficking regulator'],EPPEQRMAA,EPPE-QRMAA,0.04,0.04,38,5.943177,/Users/zacharymcgrath/Desktop/nod2 data/NOD2_E...
